In [ ]:
#---#| default_exp rescore.fdr

# FDR

Functionalities to calculate FDR.

> In peptdeep dataframes, we refer fdr values as q_values without loss of generacity.

In [ ]:
from peptdeep.rescore.fdr import *

In [ ]:
df = pd.DataFrame(
    {
        'score': np.random.random(500)*10+11,
        'decoy': 0,
        'kind': True,
    }
)
f_score = np.random.random(500)*9.9
df = pd.concat((df, 
    pd.DataFrame(
        {
            'score': f_score+0.01,
            'decoy': 1,
            'kind': False
        }
    )), ignore_index=True
)
df = pd.concat((df,
    pd.DataFrame(
        {
            'score': f_score,
            'decoy': 0,
            'kind': False
        }
    )),ignore_index=True
)
df = pd.concat((df,
    pd.DataFrame(
        {
            'score': np.random.random(5)+10,
            'decoy': 1,
            'kind': False
        }
    )), ignore_index=True
)

df = calc_fdr(df, 'score', 'decoy')
df

,score,decoy,kind,fdr
347,20.999467,0,True,0.000000
366,20.978207,0,True,0.000000
266,20.976567,0,True,0.000000
498,20.900502,0,True,0.000000
213,20.893660,0,True,0.000000
...,...,...,...,...
1216,0.064770,0,False,0.504008
669,0.062011,1,False,0.504505
1169,0.052011,0,False,0.504505
614,0.038895,1,False,0.505000


In [ ]:
df[(df.fdr < 0.01)&(df.decoy==0)]

,score,decoy,kind,fdr
347,20.999467,0,True,0.0
366,20.978207,0,True,0.0
266,20.976567,0,True,0.0
498,20.900502,0,True,0.0
213,20.893660,0,True,0.0
...,...,...,...,...
312,11.078605,0,True,0.0
493,11.052031,0,True,0.0
487,11.047116,0,True,0.0
363,11.044195,0,True,0.0


In [ ]:
#| hide
assert len(df[(df.fdr < 0.01)&(df.decoy==0)]) == 500

In [ ]:
#| hide
dff = pd.DataFrame(
    {
        'score': np.random.random(500)*10+11,
        'decoy': 0
    }
)
f_score = np.random.random(500)*9.9
dff = pd.concat((dff,
    pd.DataFrame(
        {
            'score': f_score+0.01,
            'decoy': 1
        }
    )), ignore_index=True
)
dff = pd.concat((dff,
    pd.DataFrame(
        {
            'score': f_score,
            'decoy': 0
        }
    )), ignore_index=True
)
dff = pd.concat((dff,
    pd.DataFrame(
        {
            'score': np.random.random(5)+10,
            'decoy': 1
        }
    )), ignore_index=True
)

dff['fdr'] = fdr_from_ref(dff.score.values, df.score.values, df.fdr.values)

assert len(dff[(dff.fdr < 0.01)&(dff.decoy==0)]) == 500

In [ ]:
#| hide
dff = calc_fdr_from_ref(dff, df.score.values, df.fdr.values, 'score')
assert len(dff[(dff.fdr < 0.01)&(dff.decoy==0)]) == 500